In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import urllib
import requests     # 2.18.4
import json         # 2.0.9
import pyodbc

from IPython.display import IFrame
#from sqlalchemy import create_engine

<h2> Acesso a API

In [3]:
def readJson(package):
    # Base url for package information. This is always the same.
    base_url = 'http://dados.ufop.br/api/action/datastore_search?id='

    # Construct the url for the package of interest
    package_information_url = base_url + package

    # Make the HTTP request
    package_information = requests.get(package_information_url)

    # Use the json module to load CKAN's response into a dictionary
    package_dict = json.loads(package_information.content)

    # Check the contents of the response.
    assert package_dict['success'] is True  # again make sure if response is OK
    package_dict = package_dict['result']   # we only need the 'result' part from the dictionary

    readJson.package_dict_records = package_dict['records']
    readJson.total = package_dict['total']

    #print(package_dict)           # pretty print the package information to screen

In [4]:
def readDf(package):

    #Get Total
    readJson(package)
    
    packageFull = package + '&limit='+ str(readJson.total)

    readJson(packageFull)
    
    readDf.df = pd.DataFrame(readJson.package_dict_records)

In [6]:
#Disciplinas por Curso
dispCurso =   ['2a1373db-cddb-42dc-8905-c4b084d9e289'
            ,'29502892-6bb2-49dd-99b5-b4e8e54ec056'
            ,'90435db9-846c-4a73-8721-17b8cdbbc694'
            ,'17d1874b-a54e-4137-bc18-05c7e6c59baf'
            ,'6b172a02-01ed-49ca-ae2d-1a331876b401'
            ,'be7b70be-14c1-48d4-bfe7-9c8804b136ff'
            ,'b4eb17d6-b210-4a58-a7a5-e20d9ceefab1'
            ,'9a6a533e-8bec-4b40-84f0-4d2f6f5d003a'
            ,'76f61a98-b9bc-46e9-b77b-f538f9769138']

In [7]:
#Docentes Ativos 2022
docentesAtivos = ['04e65338-1b7f-45b7-893b-05470d17dcad']

In [8]:
#Iniciação Cientifica
iniciacaoCien = ['266c947b-132c-4b49-aacd-956541043313']

In [9]:
#Evasões
evasoes = ['6dcc5cca-e3d2-4ce2-8aa0-f6066e289e2b'
        ,'2c90a54d-d9dc-4f70-bf33-346ca0fca964'
        ,'9430facb-17ad-45c6-90cf-fd666143852f'
        ,'d146d608-de81-4315-914a-41ad17ea63bf'
        ,'05c23dfb-d93f-4817-b27a-35c685cc47ea'
        ,'eab3fbd2-7189-49d8-831e-b7eeaeb162d9'
#        ,'1016c011-d8c6-40e7-be28-45bc573607ba'
        ,'cefaf737-26d2-41b7-b73c-41a46fdca2a0'
        ,'04fa169b-e174-4dab-a1a3-ea024de3bcdb']

In [10]:
#Diplomados
diplomados = ['f7117c86-4678-4005-a491-97a2aa0ede91'
            ,'fdc9918a-9e00-4ced-b3dc-bf3205c38c3b'
            ,'dfc39e36-545e-4d9c-841e-b6fccd6612eb'
            ,'1d109cf3-3638-4de3-af1d-c91be17707db'
            ,'70169dac-6b18-46a5-81e6-22b7a407f78b'
            ,'fed779db-1e1d-4677-a8bf-aefbc165f519'
            ,'e8dc65d9-e3b6-46cb-946d-6fe6300a4235'
            ,'e39aa2d8-b457-4266-9e2b-5fe19cecee58'
            ,'232c2b99-330d-441a-904a-77405a0864b8'
            ,'6850a3a1-7120-47a8-b1f5-62339ff01846']

#Diplomados apresneta 3 padrões de planilha

<h2> Limpeza dos dados

### Disciplinas

In [11]:
dataframes_list = []

for i in range (len(dispCurso)):

    readDf(dispCurso[i])
    
    dataframes_list.append(readDf.df)

df_dispCurso = (pd.concat(dataframes_list)).reset_index(drop = True)
df_dispCurso.tail()

ConnectionError: HTTPConnectionPool(host='dados.ufop.br', port=80): Max retries exceeded with url: /api/action/datastore_search?id=2a1373db-cddb-42dc-8905-c4b084d9e289 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002C8507869A0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [13]:
#Necessário criar uma variável tipo "data" contendo o ano e semestre
df_dispCurso.loc[df_dispCurso.Semestre == '2', 'Semestre'] = 7
df_dispCurso['anoSemestre'] = df_dispCurso['Ano'].astype(str) + "/" + df_dispCurso['Semestre'].astype(str)

#Alteração do formato
df_dispCurso['anoSemestre'] = pd.to_datetime(df_dispCurso['anoSemestre'],format = '%Y/%m').dt.strftime('%Y/%m')

#Ordeno, elimino colunas indesejadas e reinicio o index
df_dispCurso = (df_dispCurso.sort_values('anoSemestre').drop(columns={"Semestre","Ano"})).reset_index(drop = True)

#Renomeação
df_dispCurso['Unidade Acadêmica'] = df_dispCurso['Unidade Acadêmica'].replace(['INSTITUTO DE CIENCIAS SOCIAIS APLICADAS', 'INSTITUTO DE CIENCIAS HUMANAS E SOCIAIS', 'INSTITUTO DE CIENCIAS EXATAS E BIOLOGICAS','INSTITUTO DE CIENCIAS EXATAS E APLICADAS','INSTITUTO DE FILOSOFIA, ARTES E CULTURA','ESCOLA DE DIREITO, TURISMO E MUSEOLOGIA'],['ICSA', 'ICHS', 'ICEB', 'ICEA', 'IFAC','EDTM'])

#Adequação da Tipagem dos dados
df_dispCurso = df_dispCurso.fillna(0)
df_dispCurso = df_dispCurso.astype({'Aprovados': 'int', 'Matriculados': 'int', 'Reprovados': 'int', 'Reprovações': 'int', 'Trancamentos': 'int' })
df_dispCurso['Reprovados'] = df_dispCurso['Reprovados'] + df_dispCurso['Reprovações']
df_dispCurso = df_dispCurso.drop('Reprovações', axis=1)

#Remoção de Unidades indesejadas
df_dispCurso = df_dispCurso[((df_dispCurso['Unidade Acadêmica'] != 'UNIDADE CURSO SEQUENCIAL') & (df_dispCurso['Unidade Acadêmica'] != 'CENTRO DE EDUCACAO ABERTA E A DISTANCIA') & (df_dispCurso['Unidade Acadêmica'] != 'REITORIA'))]

#Remoção de espaços em branco indevidos
df_dispCurso = df_dispCurso.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [14]:
df_dispCurso

,Aprovados,Disciplina,Matriculados,Unidade Acadêmica,Código Disciplina,Departamento,Reprovados,_id,Trancamentos,anoSemestre
1,1,FISICA TERMICA,3,ICEB,FIS210,DEPARTAMENTO DE FISICA,2,990,0,2010/01
2,30,MECANICA CLASSICA,44,ICEB,FIS209,DEPARTAMENTO DE FISICA,14,989,0,2010/01
3,24,FISICA IV,32,ICEB,FIS133,DEPARTAMENTO DE FISICA,5,988,3,2010/01
4,16,FISICA IV,30,ICEB,FIS133,DEPARTAMENTO DE FISICA,12,987,2,2010/01
5,25,FISICA IV,29,ICEB,FIS133,DEPARTAMENTO DE FISICA,3,986,1,2010/01
...,...,...,...,...,...,...,...,...,...,...
57539,17,TRABALHO FINAL DE CURSO II (MONOGRAFIA),29,ESCOLA DE MINAS,MEC381,DEPARTAMENTO DE ENGENHARIA MECANICA,11,601,1,2021/07
57540,24,TRABALHO FINAL DE GRADUACAO I (PROJETO),35,ESCOLA DE MINAS,MEC380,DEPARTAMENTO DE ENGENHARIA MECANICA,6,600,5,2021/07
57541,30,TRANSFERENCIA E CALOR E MASSA I M,60,ESCOLA DE MINAS,MEC174,DEPARTAMENTO DE ENGENHARIA MECANICA,25,598,5,2021/07
57542,9,SISTEMAS TERMICOS,15,ESCOLA DE MINAS,MEC171,DEPARTAMENTO DE ENGENHARIA MECANICA,5,597,1,2021/07


### Docentes

In [ ]:
readDf(docentesAtivos[0])

docentes = readDf.df

docentes = docentes[['Nome do Servidor','Escolaridade','setor lotacao','Sexo','raca','Jornada']]

In [ ]:
dep_Doc = docentes['setor lotacao'].unique()
dep_Alu = disciplinas['Departamento'].unique()

In [ ]:
(dep_Alu == dep_Doc)

### Diplomados

In [63]:
dataframes_list = []

for i in range (len(diplomados)):

    readDf(diplomados[i])
    
    dataframes_list.append(readDf.df)
    
    if i == 0:
        dataframes_list[i] = dataframes_list[i].rename(columns={"curso":"Descrição Curso","unidade_academica":"Unidade Acadêmica","diplomados":"Diplomados","ano":"Ano","formato":"Formato","semestre":"Semestre","habilitacao":"Descrição Habilitação","campus":"Campus"})
    elif ((i == 6) | (i == 8)):
        firstRow = pd.DataFrame(dataframes_list[i].columns)
        dataframes_list[i] = (pd.concat([dataframes_list[i].columns.to_frame().T, dataframes_list[i]], ignore_index=True).set_axis(range(len(dataframes_list[i].columns)), axis=1))
        dataframes_list[i].columns = ['Descrição Curso','Campus','Diplomados','Semestre','Ano','Unidade Acadêmica','_id','Formato']
    elif i == 7:
        firstRow = pd.DataFrame(dataframes_list[i].columns)
        dataframes_list[i] = (pd.concat([dataframes_list[i].columns.to_frame().T, dataframes_list[i]], ignore_index=True).set_axis(range(len(dataframes_list[i].columns)), axis=1))
        dataframes_list[i].columns = ['Descrição Curso','Campus','Semestre','Ano','Diplomados','Unidade Acadêmica','_id','Formato']
        
    elif i == 9:
        firstRow = pd.DataFrame(dataframes_list[i].columns)
        dataframes_list[i] = (pd.concat([dataframes_list[i].columns.to_frame().T, dataframes_list[i]], ignore_index=True).set_axis(range(len(dataframes_list[i].columns)), axis=1))
        dataframes_list[i].columns = ['Diplomados','Descrição Curso','Campus','Semestre','Ano','Unidade Acadêmica','_id','Formato']

for i in range(len(diplomados)):
    if i < 6:
        dataframes_list[i] = dataframes_list[i].drop(columns=['Descrição Habilitação','_id','Unidade Acadêmica'])
    else:
        dataframes_list[i] = dataframes_list[i].drop(columns=['_id','Unidade Acadêmica'])

df_diplomados = (pd.concat(dataframes_list)).reset_index(drop = True)

#Necessário criar uma variável tipo "data" contendo o ano e semestre
df_diplomados.loc[df_diplomados.Semestre == '2', 'Semestre'] = 7
df_diplomados['anoSemestre'] = df_diplomados['Ano'].astype(str) + "/" + df_diplomados['Semestre'].astype(str)

#Alteração do formato
df_diplomados['anoSemestre'] = pd.to_datetime(df_diplomados['anoSemestre'],format = '%Y/%m').dt.strftime('%Y/%m')

#Ordeno, elimino colunas indesejadas e reinicio o index
df_diplomados = (df_diplomados.sort_values('anoSemestre').drop(columns={"Semestre","Ano"})).reset_index(drop = True)

In [64]:
df_diplomados

,Formato,Descrição Curso,Diplomados,Campus,anoSemestre
0,PRESENCIAL,HISTORIA-HIS,12,MARIANA,2000/01
1,PRESENCIAL,FILOSOFIA-FIL,6,OURO PRETO,2000/01
2,PRESENCIAL,FILOSOFIA-FIL,4,OURO PRETO,2000/01
3,PRESENCIAL,NUTRICAO-NUT,27,OURO PRETO,2000/01
4,PRESENCIAL,ENGENHARIA METALURGICA-MET,13,OURO PRETO,2000/01
...,...,...,...,...,...
1702,PRESENCIAL,ENGENHARIA DE PRODUCAO-PJM,12,JOÃO MONLEVADE,2021/07
1703,PRESENCIAL,ENGENHARIA DE PRODUCAO-PRO,13,OURO PRETO,2021/07
1704,PRESENCIAL,ENGENHARIA ELETRICA-EJM,16,JOÃO MONLEVADE,2021/07
1705,PRESENCIAL,ENGENHARIA MECANICA-MEC,15,OURO PRETO,2021/07


### Evasões

In [69]:
dataframes_list = []

for i in range (len(evasoes)):

    readDf(evasoes[i])
    
    dataframes_list.append(readDf.df)

#     if i == 0:
#         dataframes_list[i] = dataframes_list[i].rename(columns={"curso":"Descrição Curso","unidade_academica":"Unidade Acadêmica","diplomados":"Diplomados","ano":"Ano","formato":"Formato","semestre":"Semestre","habilitacao":"Descrição Habilitação","campus":"Campus"})
#     elif i >= 6:
#         firstRow = pd.DataFrame(dataframes_list[i].columns)
#         dataframes_list[i] = (pd.concat([dataframes_list[i].columns.to_frame().T, dataframes_list[i]], ignore_index=True).set_axis(range(len(dataframes_list[i].columns)), axis=1))
#         dataframes_list[i].columns = ['Diplomados','Descrição Curso','Campus','Semestre','Ano','Unidade Acadêmica','_id','Formato']

# for i in range(len(diplomados)):
#     if i < 6:
#         dataframes_list[i] = dataframes_list[i].drop(columns=['Descrição Habilitação','_id','Unidade Acadêmica'])
#     else:
#         dataframes_list[i] = dataframes_list[i].drop(columns=['_id','Unidade Acadêmica'])

df_evasoes = (pd.concat(dataframes_list)).reset_index(drop = True)
df_evasoes

,semestre,formato,curso,tipo_evasao,ano,evadidos,modalidade,_id,unidade_academica,campus,...,1,3,2020,OURO PRETO,5,2021,13,ADMINISTRACAO PUBLICA-D06,2,BACHARELADO
0,2,DISTANCIA,LICENCIATURA EM EDUCACAO BASICA - ANOS INICIAI...,CANCELAMENTO,2000,7,LICENCIATURA,1,CENTRO DE EDUCACAO ABERTA E A DISTANCIA,OURO PRETO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,PRESENCIAL,CIENCIAS BIOLOGICAS-CBI,CANCELAMENTO,2000,1,,2,INSTITUTO DE CIENCIAS EXATAS E BIOLOGICAS,OURO PRETO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,PRESENCIAL,LETRAS-LET,NRM,2001,1,,3,INSTITUTO DE CIENCIAS HUMANAS E SOCIAIS,MARIANA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,DISTANCIA,LICENCIATURA EM EDUCACAO BASICA - ANOS INICIAI...,CANCELAMENTO,2001,1,LICENCIATURA,4,CENTRO DE EDUCACAO ABERTA E A DISTANCIA,OURO PRETO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,PRESENCIAL,CIENCIAS BIOLOGICAS-CBI,NRM,2001,1,,5,INSTITUTO DE CIENCIAS EXATAS E BIOLOGICAS,OURO PRETO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99,NaN,NaN,...,NaN,NaN,NaN,OURO PRETO,NaN,2021,3,ESTATISTICA-EST,2,BACHARELADO
4280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,NaN,NaN,...,NaN,NaN,NaN,OURO PRETO,NaN,2021,1,FARMACIA-FAR,2,BACHARELADO
4281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101,NaN,NaN,...,NaN,NaN,NaN,OURO PRETO,NaN,2021,1,FISICA-FSB,2,BACHARELADO
4282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102,NaN,NaN,...,NaN,NaN,NaN,OURO PRETO,NaN,2021,1,NUTRICAO-NUT,2,BACHARELADO


### Iniciação Cientifica

In [19]:
readDf(iniciacaoCien[0])

iniciacao = readDf.df

In [20]:
iniciacao

,orientador,tipo_bolsa,fomento,ano,título,curso_bolsista,bolsista,programa,id_projeto,setor,_id,linha_pesquisa
0,CLAUDIA MARTINS CARNEIRO,BOLSISTA,FAPEMIG,2017,Avaliação da eficácia de novas estratégias ter...,FARMACIA,ADILSON DE BARROS MARTINS,PROGRAMA DE BOLSAS DE INICIAÇÃO CIENTÍFICA E T...,2590,DEACL,1,
1,PAULA MELO DE ABREU VIEIRA,BOLSISTA,FAPEMIG,2017,Quantificação do parasistismo no trato gastroi...,FARMACIA,ALINE TONHELA FERRAZ,PROGRAMA DE BOLSAS DE INICIAÇÃO CIENTÍFICA E T...,2487,DECBI,2,OBJETIVOS: NESTA LINHA DE PESQUISA TEMOS COMO ...
2,HARRISON BACHION CERIBELI,VOLUNTÁRIO,UFOP,2017,Produtos e serviços turísticos para a criança ...,ADMINISTRACAO,AMANDA ALVES CAMPOS,PROGRAMA INSTITUCIONAL DE VOLUNTÁRIOS DE INICI...,2626,DECEG,3,EMENTA: MÚLTIPLAS DIMENSÕES DO PROCESSO ESTRAT...
3,LUCCAS ASSIS ATTILIO,VOLUNTÁRIO,UFOP,2017,Financeirização e crescimento econômico brasil...,CIENCIAS ECONOMICAS,ANA CARMEM GOMES MENEZES,PROGRAMA INSTITUCIONAL DE VOLUNTÁRIOS DE INICI...,2774,DECEG,4,EMENTA: O OBJETIVO DESSA LINHA DE PESQUISA É D...
4,MARCELO VIANA RAMOS,VOLUNTÁRIO,UFOP,2017,URBANIZAÇÃO TURÍSTICA NO CONTEXTO DOS MEGA-EVE...,TURISMO,ANA CAROLINA DA SILVA,PROGRAMA INSTITUCIONAL DE VOLUNTÁRIOS DE INICI...,3200,DETUR,5,GESTÃO E PLANEJAMENTO DO TURISMO
...,...,...,...,...,...,...,...,...,...,...,...,...
3786,MARCO ANTONIO MELO FRANCO (DEEDU),BOLSISTA,CNPQ,2021,COVID-19 E ESCOLARIZAÇÃO: O QUE MUDA NO PROCES...,ENGENHARIA DE PRODUCAO,FABIO ROQUE RIBEIRO,PROGRAMA INSTITUCIONAL DE BOLSAS DE INICIAÇÃO ...,7722,DEEDU,3787,TEM COMO OBJETIVO INVESTIGAR EM DIFERENTES ESP...
3787,MARCELO LOURES DOS SANTOS (DEEDU),BOLSISTA,UFOP,2021,Contribuições da Metodologia de Produções Narr...,ARTES CENICAS,PEDRO GABAN PETINDA MOREIRA,PROGRAMA DE INICIAÇÃO À PESQUISA - 1º SEMESTRE,7063,DEEDU,3788,TEM COMO OBJETIVO INVESTIGAR AS DIVERSAS PRÁTI...
3788,MARCELO LOURES DOS SANTOS (DEEDU),BOLSISTA,UFOP,2021,Contribuições da Metodologia de Produções Narr...,PEDAGOGIA,REBECA DE SILVA OLIVEIRA,PROGRAMA DE INICIAÇÃO À PESQUISA - 1º SEMESTRE,7063,DEEDU,3789,TEM COMO OBJETIVO INVESTIGAR AS DIVERSAS PRÁTI...
3789,ANELISE FONSECA DUTRA (DELET),BOLSISTA,UFOP,2021,Teletandem: o desenvolvimento da competência i...,LETRAS,FERNANDA LIMA ALVARES DA SILVA,PROGRAMA DE INICIAÇÃO À PESQUISA - 1º SEMESTRE,7045,DELET,3790,ESTA LINHA INCORPORA A LINGUÍSTICA APLICADA EM...


<h2> Exploração dos dados

In [65]:
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=DESKTOP-021P0C7;"
            "Database=DW_UFOP;"
            "Trusted_Connection=yes;")
cnxn = pyodbc.connect(cnxn_str)

DB = {'servername': 'DESKTOP-021P0C7',
      'database': 'DW_UFOP',
      'driver': 'driver=SQL Server Native Client 11.0'}

engine = create_engine('mssql+pyodbc://' + DB['servername'] + '/' + DB['database'] + "?" + DB['driver'])

In [68]:
df_diplomados.to_sql('Graduados', index=False, con=engine)